lung: 6'35''

In [172]:
import pickle
import networkx as nx
import numpy as np
import pandas as pd
# import math
# import EoN
# import random
import pickle
from Phage_api import settings_local as local_settings
import glob
# import gseapy as gp
import os.path

In [173]:
import importlib.util
gseapy=importlib.util.spec_from_file_location("gseapy","/home/platform/project/crustdb_platform/crustdb_api/workspace/analysis_script/scripts/cytotopo_reference_package/gseapy/plot.py")
DotPlot = importlib.util.module_from_spec(gseapy)
gseapy.loader.exec_module(DotPlot)

In [174]:
home = local_settings.CRUSTDB_DATABASE+'topology/'
home

'/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/'

In [175]:
def run_go_analysis(go_df):
    result_new = go_df
    if result_new.shape[0] > 200:
        figsize=False
        size = False
        top_term=3
    else:
        figsize = (5,10)
        size = 50
        top_term = 5

    x_key = 'Gene_set'
    if not figsize:
        figsize = (int(len(result_new[x_key].unique())/3), int(top_term*len(result_new[x_key].unique())/10))
    if not size:
        size = int(top_term*len(result_new[x_key].unique())/15)

    result_new.loc[:,'Term'] = result_new.loc[:,'Term'].str.split('(', expand=True)[0]

    cutoff = 0.01
    phage1 = True
    _check = int(len(result_new[result_new.loc[:,'Adjusted P-value'] <= cutoff][x_key].unique()))
    while _check < 3:
        # cutoff = cutoff + 0.01
        if phage1:
            cutoff *= 10
        else:
            cutoff += 0.01
        _check = int(len(result_new[result_new.loc[:,'Adjusted P-value'] <= cutoff][x_key].unique()))
        if phage1 and _check >= 3:
            cutoff /= 10
            phage1 = False
        if np.isinf(cutoff):
            cutoff = 0.01
            break
    
    dot = DotPlot.DotPlot(
        df=result_new,
        x='Gene_set',
        y='Term',
        x_order=False,
        y_order=False,
        hue="Adjusted P-value",
        title="title",
        thresh=cutoff,
        n_terms=int(top_term),
        dot_scale=size,
        figsize=figsize,
        cmap="viridis_r",
        ofname=None,
        marker='o',
    )
    go_info = dot.data[['Gene_set', 'Term', 'p_inv', 'Hits_ratio']]
    return go_info

In [176]:
def generate_go_info(species, conformation, algo, param):
    fname = home+species+'/'+conformation+'/'+algo+'/'+param+'/Go.csv'
    if not os.path.isfile(fname):
        print('Go.csv does not exist:', fname)
        return
    go_df = pd.read_csv(fname, index_col=0)
    go_info = run_go_analysis(go_df)
    go_info.to_csv(home+species+'/'+conformation+'/'+algo+'/'+param+'/Go_result.csv')


### 应该跑的部分

In [177]:
# layer1 = glob.glob(home+'*')
# error_entries = []
# for _species in layer1:
#     species = _species.split('/')[-1]
#     if species not in ['Lung']:
#         continue
#     layer2 = glob.glob(_species+'/*')
#     for _conformation in layer2:
#         conformation = _conformation.split('/')[-1]
#         try:
#             for algo in ['1NN', 'KNN', 'KNN_SNN', 'RNN', 'RNN_SNN']:
#                 if algo in ['1NN']:
#                     param = '/'
#                     generate_go_info(species, conformation, algo, param)
#                 else:
#                     _algo = _conformation + '/' + algo
#                     layer3 = glob.glob(_algo + '/*')
#                     for _param in layer3:
#                         if 'pkl' in _param or 'csv' in _param:
#                             continue
#                         param = _param.split('/')[-1]
#                         generate_go_info(species, conformation, algo, param)
#         except:
#             print('Error ---> '+species+'/'+conformation+'/'+algo+'/'+param)
#             error_entries.append([species, conformation, algo, param])
#     print('species ends', species)

### 针对本次出错的部分，单独跑的一部分代码（不适用于以后）

In [178]:
error_entries2 = [
    "Lung/Lung5_Rep1.gem.txt.cell_type.fibroblast_ISV2/1NN",
    "Lung/Lung5_Rep3.gem.txt.cell_type.T_CD8_memory_2R7L/1NN",
    "Lung/Lung12.gem.txt.cell_type.tumor_9_RRH9/KNN",
    "Lung/Lung5_Rep2.gem.txt.cell_type.tumor_12_7PVN/1NN",
    "Lung/Lung5_Rep2.gem.txt.cell_type.endothelial_7PVN/RNN",
    "Lung/Lung5_Rep3.gem.txt.cell_type.tumor_12_2R7L/1NN",
    "Lung/Lung5_Rep1.gem.txt.cell_type.tumor_13_ISV2/1NN",
    "Lung/Lung13.gem.txt.cell_type.T_CD8_memory_Q6PL/KNN",
    "Lung/Lung13.gem.txt.cell_type.epithelial_Q6PL/KNN",
    "Lung/Lung13.gem.txt.cell_type.tumor_6_Q6PL/KNN_SNN",
    "Lung/Lung9_Rep1.gem.txt.cell_type.epithelial_FCSY/RNN_SNN",
    "Lung/Lung9_Rep1.gem.txt.cell_type.NK_FCSY/RNN_SNN",
    "Lung/Lung9_Rep2.gem.txt.cell_type.endothelial_HF1K/RNN",
    "Lung/Lung9_Rep2.gem.txt.cell_type.monocyte_HF1K/RNN",
    "Lung/Lung9_Rep2.gem.txt.cell_type.tumor_13_HF1K/RNN",
    "Lung/Lung9_Rep2.gem.txt.cell_type.macrophage_HF1K/RNN",
    "Lung/Lung6.gem.txt.cell_type.T_CD8_memory_W682/KNN",
]
layer1 = glob.glob(home+'*')
error_entries = []
for _species in layer1:
    species = _species.split('/')[-1]
    if species not in ['Lung']:
        continue
    layer2 = glob.glob(_species+'/*')
    for _conformation in layer2:
        conformation = _conformation.split('/')[-1]
        try:
            for algo in ['1NN', 'KNN', 'KNN_SNN', 'RNN', 'RNN_SNN']:
                if algo in ['1NN']:
                    param = '/'
                    if species+'/'+conformation+'/'+algo in error_entries2:
                        generate_go_info(species, conformation, algo, param)
                else:
                    _algo = _conformation + '/' + algo
                    layer3 = glob.glob(_algo + '/*')
                    for _param in layer3:
                        if 'pkl' in _param or 'csv' in _param:
                            continue
                        param = _param.split('/')[-1]
                        if species+'/'+conformation+'/'+algo in error_entries2:
                            generate_go_info(species, conformation, algo, param)
        except:
            print('Error ---> '+species+'/'+conformation+'/'+algo+'/'+param)
            error_entries.append([species, conformation, algo, param])
    print('species ends', species)

Error ---> Lung/Lung5_Rep1.gem.txt.cell_type.fibroblast_ISV2/1NN//
Error ---> Lung/Lung5_Rep3.gem.txt.cell_type.T_CD8_memory_2R7L/1NN//


Error ---> Lung/Lung12.gem.txt.cell_type.tumor_9_RRH9/KNN/5
Error ---> Lung/Lung5_Rep2.gem.txt.cell_type.tumor_12_7PVN/1NN//
Go.csv does not exist: /home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/Lung/Lung5_Rep2.gem.txt.cell_type.endothelial_7PVN/RNN/0.15/Go.csv
Error ---> Lung/Lung5_Rep3.gem.txt.cell_type.tumor_12_2R7L/1NN//
Error ---> Lung/Lung5_Rep1.gem.txt.cell_type.tumor_13_ISV2/1NN//
Go.csv does not exist: /home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/Lung/Lung13.gem.txt.cell_type.T_CD8_memory_Q6PL/KNN/10/Go.csv
Go.csv does not exist: /home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/Lung/Lung13.gem.txt.cell_type.epithelial_Q6PL/KNN/5/Go.csv
Go.csv does not exist: /home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/Lung/Lung13.gem.txt.cell_type.tumor_6_Q6PL/KNN_SNN/15/Go.csv
Go.csv does not exist: /home/platform/project/crustd

In [179]:
generate_go_info('Lung', 'Lung5_Rep2.gem.txt.cell_type.tumor_12_7PVN', '1NN', '/')

ValueError: Warning: No enrich terms when cutoff = 0.01